#Depedencies and Setup

In [53]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import faiss
import random
import time
import statistics

In [5]:
import zipfile
with zipfile.ZipFile("wher your zip file is", 'r') as zip_ref:
    zip_ref.extractall("archive")

In [6]:
flowers = pathlib.Path("archive/flowers")

In [7]:
data = tf.keras.preprocessing.image_dataset_from_directory(
  flowers,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(256, 256),
  batch_size=32)


valdata = tf.keras.preprocessing.image_dataset_from_directory(
  flowers,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(256, 256),
  batch_size=32)




Found 4317 files belonging to 5 classes.
Using 3454 files for training.
Found 4317 files belonging to 5 classes.
Using 863 files for validation.


In [8]:
class_names = data.class_names

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
data = data.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
valdata = valdata.cache().prefetch(buffer_size=AUTOTUNE)

#Train Model

In [ ]:
epochs = 50
path_checkpoint = "model_checkpoint.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(256, 256,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(256, 256, 3),
    include_top=False,
)  

base_model.trainable = False

inputs = keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(26)(x)
model = keras.Model(inputs, outputs)


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
history = model.fit(
  data,
  epochs=epochs,
  validation_data= valdata,
  callbacks=[es_callback, modelckpt_callback]
)

Epoch 1/50
108/108 [==============================] - ETA: 0s - loss: 0.8913 - accuracy: 0.7102
Epoch 1: val_loss improved from inf to 0.50331, saving model to model_checkpoint.h5
108/108 [==============================] - 44s 290ms/step - loss: 0.8913 - accuracy: 0.7102 - val_loss: 0.5033 - val_accuracy: 0.8320
Epoch 2/50
108/108 [==============================] - ETA: 0s - loss: 0.4623 - accuracy: 0.8509
Epoch 2: val_loss improved from 0.50331 to 0.41908, saving model to model_checkpoint.h5
108/108 [==============================] - 26s 242ms/step - loss: 0.4623 - accuracy: 0.8509 - val_loss: 0.4191 - val_accuracy: 0.8505
Epoch 3/50
108/108 [==============================] - ETA: 0s - loss: 0.4003 - accuracy: 0.8662
Epoch 3: val_loss improved from 0.41908 to 0.38630, saving model to model_checkpoint.h5
108/108 [==============================] - 27s 252ms/step - loss: 0.4003 - accuracy: 0.8662 - val_loss: 0.3863 - val_accuracy: 0.8598
Epoch 4/50
108/108 [==============================

In [ ]:
base_model.trainable = True
model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
  data,
  epochs=epochs,
  validation_data= valdata,
  callbacks=[es_callback, modelckpt_callback]
)

Epoch 1/50
108/108 [==============================] - ETA: 0s - loss: 0.1760 - accuracy: 0.9404
Epoch 1: val_loss improved from 0.31944 to 0.29756, saving model to model_checkpoint.h5
108/108 [==============================] - 105s 905ms/step - loss: 0.1760 - accuracy: 0.9404 - val_loss: 0.2976 - val_accuracy: 0.9096
Epoch 2/50
108/108 [==============================] - ETA: 0s - loss: 0.1391 - accuracy: 0.9551
Epoch 2: val_loss improved from 0.29756 to 0.28524, saving model to model_checkpoint.h5
108/108 [==============================] - 94s 869ms/step - loss: 0.1391 - accuracy: 0.9551 - val_loss: 0.2852 - val_accuracy: 0.9143
Epoch 3/50
108/108 [==============================] - ETA: 0s - loss: 0.0984 - accuracy: 0.9653
Epoch 3: val_loss did not improve from 0.28524
108/108 [==============================] - 94s 867ms/step - loss: 0.0984 - accuracy: 0.9653 - val_loss: 0.2932 - val_accuracy: 0.9177
Epoch 4/50
108/108 [==============================] - ETA: 0s - loss: 0.0803 - accurac

In [ ]:
model.save('flowers.h5')

In [ ]:
model.save('flowers')

In [ ]:
!zip -r flowers.zip /content/flowers

  adding: content/flowers/ (stored 0%)
  adding: content/flowers/variables/ (stored 0%)
  adding: content/flowers/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/flowers/variables/variables.index (deflated 79%)
  adding: content/flowers/assets/ (stored 0%)
  adding: content/flowers/saved_model.pb (deflated 91%)
  adding: content/flowers/keras_metadata.pb (deflated 96%)
